In [1]:
import pandas as pd
import markdown
import io
import re
from dateutil import parser
import calendar
from datetime import date

In [69]:
import re
import calendar
from datetime import date
from dateutil import parser
import pandas as pd

# -----------------------------
# CONFIG
# -----------------------------
MAX_SCAN_ROWS = 6

TARGET_DATES = {
    "this_quarter": date(2025, 9, 30),
    "previous_quarter": date(2025, 6, 30),
    "same_q_last_year": date(2024, 9, 30)
}

# -----------------------------
# REGEX - ADDED MONTH-YEAR PATTERN
# -----------------------------
FULL_DATE_REGEX = re.compile(r'''
    \d{1,2}[./-]\d{1,2}[./-]\d{2,4}                     # 30.09.2025
    |
    \d{1,2}\s*[-]\s*(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Sept|Oct|Nov|Dec)[a-z]*\s*[-]\s*\d{2,4}  # 30-Sep-25
    |
    \d{1,2}(?:st|nd|rd|th)?\s+(January|February|March|April|May|June|July|August|September|October|November|December)\s*,?\s*\d{4}  # 30th September, 2025
    |
    (January|February|March|April|May|June|July|August|September|October|November|December)\s+\d{1,2}(?:st|nd|rd|th)?\s*,?\s*\d{4}  # September 30th, 2025
''', re.IGNORECASE | re.VERBOSE)

# NEW: Month-Year only pattern
MONTH_YEAR_REGEX = re.compile(r'''
    (Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Sept|Oct|Nov|Dec)[a-z]*\s*,?\s*\d{4}  # Sept, 2025 or Sept 2025
    |
    (January|February|March|April|May|June|July|August|September|October|November|December)\s*,?\s*\d{4}  # September, 2025
''', re.IGNORECASE | re.VERBOSE)

MONTH_MAP = {
    'jan': 1, 'january': 1, 'feb': 2, 'february': 2, 'mar': 3, 'march': 3,
    'apr': 4, 'april': 4, 'may': 5, 'jun': 6, 'june': 6, 'jul': 7, 'july': 7,
    'aug': 8, 'august': 8, 'sep': 9, 'sept': 9, 'september': 9,
    'oct': 10, 'october': 10, 'nov': 11, 'november': 11, 'dec': 12, 'december': 12
}

# -----------------------------
# CLEANING
# -----------------------------
def clean_text(text: str) -> str:
    """Clean text while preserving date information"""
    if not isinstance(text, str):
        text = str(text)
    
    # Handle typos
    text = re.sub(r'3lst', '31st', text)
    text = re.sub(r'(\d)lst\b', r'\1st', text)
    
    # Normalize whitespace
    text = text.replace('\n', ' ').replace('\t', ' ')
    
    # Remove markdown and formatting
    text = re.sub(r'\*\*|__|`', '', text)
    
    # Remove audit annotations
    text = re.sub(r'\(?\s*(Unaudited|Audited)\s*\)?', ' ', text, flags=re.I)
    
    # Remove extra phrases (but keep ordinal suffixes for date extraction)
    text = re.sub(r'Quarter ended|Six Months ended|Year ended|ended|Refer note \d+', ' ', text, flags=re.I)
    
    # Normalize spaces
    text = re.sub(r'\s+', ' ', text)
    
    return text.strip()

# -----------------------------
# DATE EXTRACTION
# -----------------------------
def extract_all_date_strings(text: str):
    """Extract all potential date strings from text"""
    dates = []
    
    # First try to find full dates (with day)
    for match in FULL_DATE_REGEX.finditer(text):
        dates.append(match.group(0).strip())
    
    # If no full dates found, try month-year only
    if not dates:
        for match in MONTH_YEAR_REGEX.finditer(text):
            dates.append(match.group(0).strip())
    
    return dates

# -----------------------------
# DATE PARSING
# -----------------------------
def parse_date(date_str: str):
    """Parse date string to date object"""
    if not date_str:
        return None
    
    try:
        # Remove ordinal suffixes before parsing
        date_str_clean = re.sub(r'(\d+)(st|nd|rd|th)\b', r'\1', date_str, flags=re.I)
        date_str_clean = date_str_clean.strip()
        
        # Try DD.MM.YYYY or DD/MM/YYYY or DD-MM-YYYY format
        match = re.match(r'^(\d{1,2})[./-](\d{1,2})[./-](\d{2,4})$', date_str_clean)
        if match:
            day, month, year = match.groups()
            year = int(year)
            if year < 100:
                year += 2000 if year < 50 else 1900
            return date(year, int(month), int(day))
        
        # Try DD-Month-YYYY format (30-September-2025)
        match = re.match(r'^(\d{1,2})\s*[-]\s*([A-Za-z]+)\s*[-]\s*(\d{2,4})$', date_str_clean)
        if match:
            day, month_str, year = match.groups()
            month_key = month_str[:4].lower()
            month = MONTH_MAP.get(month_key) or MONTH_MAP.get(month_str.lower())
            if month:
                year = int(year)
                if year < 100:
                    year += 2000 if year < 50 else 1900
                return date(year, month, int(day))
        
        # Try DD Month YYYY format (30 September 2025 or 30 September, 2025)
        match = re.match(r'^(\d{1,2})\s+([A-Za-z]+)\s*,?\s*(\d{4})$', date_str_clean, re.I)
        if match:
            day, month_str, year = match.groups()
            month_key = month_str[:4].lower()
            month = MONTH_MAP.get(month_key) or MONTH_MAP.get(month_str.lower())
            if month:
                return date(int(year), month, int(day))
        
        # Try Month DD YYYY format (September 30 2025 or September 30, 2025)
        match = re.match(r'^([A-Za-z]+)\s+(\d{1,2})\s*,?\s*(\d{4})$', date_str_clean, re.I)
        if match:
            month_str, day, year = match.groups()
            month_key = month_str[:4].lower()
            month = MONTH_MAP.get(month_key) or MONTH_MAP.get(month_str.lower())
            if month:
                return date(int(year), month, int(day))
        
        # Try Month YYYY format (Sept, 2025 or September 2025) - infer last day of month
        match = re.match(r'^([A-Za-z]+)\s*,?\s*(\d{4})$', date_str_clean, re.I)
        if match:
            month_str, year = match.groups()
            month_key = month_str[:4].lower()
            month = MONTH_MAP.get(month_key) or MONTH_MAP.get(month_str.lower())
            if month:
                year = int(year)
                # Get last day of the month
                last_day = calendar.monthrange(year, month)[1]
                return date(year, month, last_day)
        
        # Fallback to dateutil
        dt = parser.parse(date_str_clean, dayfirst=True, fuzzy=True)
        return dt.date()
        
    except Exception as e:
        return None

# -----------------------------
# MAIN FUNCTION - SCAN HEADERS AND ROWS
# -----------------------------
def find_date_columns(df, target_dates=TARGET_DATES, max_rows=MAX_SCAN_ROWS, debug=False):
    """
    Scan column headers AND first few rows to find which column contains each target date.
    
    Args:
        df: pandas DataFrame
        target_dates: dict of {name: date_object}
        max_rows: number of rows to scan
        debug: print debug information
    
    Returns:
        dict: {date_name: column_index}
    """
    result = {}
    
    if debug:
        print(f"Scanning column headers and first {max_rows} rows...")
        print(f"Looking for dates: {target_dates}")
        print("="*70 + "\n")
    
    # STEP 1: Scan column headers
    if debug:
        print("STEP 1: Scanning column headers...")
    
    for col_idx, col_name in enumerate(df.columns):
        col_str = str(col_name)
        cleaned = clean_text(col_str)
        
        if debug:
            print(f"Column {col_idx}: '{col_str}'")
        
        date_strings = extract_all_date_strings(cleaned)
        
        if date_strings:
            if debug:
                print(f"  Found date strings: {date_strings}")
            
            for date_str in date_strings:
                parsed_date = parse_date(date_str)
                
                if debug and parsed_date:
                    print(f"  Parsed '{date_str}' → {parsed_date}")
                
                if parsed_date:
                    for date_name, target_date in target_dates.items():
                        if parsed_date == target_date and date_name not in result:
                            result[date_name] = col_idx
                            if debug:
                                print(f"  ✓ MATCH! {date_name} found in column {col_idx}")
    
    # STEP 2: Scan first few rows (for multi-row headers)
    if debug:
        print(f"\nSTEP 2: Scanning first {max_rows} rows...")
    
    for row_idx in range(min(max_rows, len(df))):
        if debug:
            print(f"\nRow {row_idx}:")
        
        for col_idx in range(len(df.columns)):
            cell = df.iloc[row_idx, col_idx]
            
            # Skip NaN and non-string cells
            if pd.isna(cell):
                continue
            
            cell_str = str(cell)
            cleaned = clean_text(cell_str)
            
            if not cleaned:
                continue
            
            # Look for dates
            date_strings = extract_all_date_strings(cleaned)
            
            if date_strings:
                if debug:
                    print(f"  Col {col_idx}: '{cell_str[:60]}...' → found {date_strings}")
                
                for date_str in date_strings:
                    parsed_date = parse_date(date_str)
                    
                    if debug and parsed_date:
                        print(f"    Parsed '{date_str}' → {parsed_date}")
                    
                    if parsed_date:
                        for date_name, target_date in target_dates.items():
                            if parsed_date == target_date and date_name not in result:
                                result[date_name] = col_idx
                                if debug:
                                    print(f"    ✓ MATCH! {date_name} found in column {col_idx}")
        
        # Early exit if all found
        if len(result) == len(target_dates):
            if debug:
                print("\nAll dates found! Stopping scan.")
            break
    
    if debug:
        print("\n" + "="*70)
        print(f"Final result: {result}\n")
    
    return result

# -----------------------------
# CONVENIENCE FUNCTION
# -----------------------------
def get_column_indices(df, max_rows=MAX_SCAN_ROWS, debug=False):
    """
    Get column indices for target dates.
    
    Returns:
        tuple: (this_quarter_column, previous_quarter_column, same_q_last_year_column)
    """
    columns = find_date_columns(df, max_rows=max_rows, debug=debug)
    
    this_quarter_column = columns.get('this_quarter')
    previous_quarter_column = columns.get('previous_quarter')
    same_q_last_year_column = columns.get('same_q_last_year')
    
    return this_quarter_column, previous_quarter_column, same_q_last_year_column

# -----------------------------
# USAGE
# -----------------------------
def get_column_index(df, debug=False):
    """Main function to get column indices"""
    
    # Find columns
    columns = find_date_columns(df, debug=debug)
    
    # Get individual variables
    this_quarter_column, previous_quarter_column, same_q_last_year_column = get_column_indices(df, debug=debug)

    return this_quarter_column, previous_quarter_column, same_q_last_year_column

# -----------------------------
# TEST
# -----------------------------
if __name__ == "__main__":
    # Test with your second table structure
    markdown_file_path = "saved_md_files/d60ccb02-681e-447e-bb12-263e2ded9547_10.md"
    # 1. Load the markdown file content
    with open(markdown_file_path, 'r') as f:
        text = f.read()
    # 2. Convert text to HTML (ensure 'tables' extension is included)
    table = markdown.markdown(text, extensions=['tables'])
    # 3. Parse HTML with Pandas
    # read_html returns a list of DataFrames, so we take the first one
    df = pd.read_html(io.StringIO(table))[0]
    #print(df.iloc[0:4,:])
    
    # Find columns
    columns = find_date_columns(df, debug=False)
    print("*********************", columns)
    
    # Get individual variables
    this_quarter_column, previous_quarter_column, same_q_last_year_column = get_column_index(df, debug=False)
    print(this_quarter_column)
    print(previous_quarter_column)
    print(same_q_last_year_column)

********************* {'this_quarter': 2, 'previous_quarter': 3, 'same_q_last_year': 4}
2
3
4


In [63]:
import re
import calendar
from datetime import date
from dateutil import parser
import pandas as pd

# -----------------------------
# CONFIG
# -----------------------------
MAX_SCAN_ROWS = 6

TARGET_DATES = {
    "this_quarter": date(2025, 9, 30),
    "previous_quarter": date(2025, 6, 30),
    "same_q_last_year": date(2024, 9, 30)
}

# -----------------------------
# REGEX - FIXED TO HANDLE "30th September, 2025"
# -----------------------------
FULL_DATE_REGEX = re.compile(r'''
    \d{1,2}[./-]\d{1,2}[./-]\d{2,4}                     # 30.09.2025
    |
    \d{1,2}\s*[-]\s*(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Sept|Oct|Nov|Dec)[a-z]*\s*[-]\s*\d{2,4}  # 30-Sep-25
    |
    \d{1,2}(?:st|nd|rd|th)?\s+(January|February|March|April|May|June|July|August|September|October|November|December)\s*,?\s*\d{4}  # 30th September, 2025
    |
    (January|February|March|April|May|June|July|August|September|October|November|December)\s+\d{1,2}(?:st|nd|rd|th)?\s*,?\s*\d{4}  # September 30th, 2025
''', re.IGNORECASE | re.VERBOSE)

MONTH_MAP = {
    'jan': 1, 'january': 1, 'feb': 2, 'february': 2, 'mar': 3, 'march': 3,
    'apr': 4, 'april': 4, 'may': 5, 'jun': 6, 'june': 6, 'jul': 7, 'july': 7,
    'aug': 8, 'august': 8, 'sep': 9, 'sept': 9, 'september': 9,
    'oct': 10, 'october': 10, 'nov': 11, 'november': 11, 'dec': 12, 'december': 12
}

# -----------------------------
# CLEANING
# -----------------------------
def clean_text(text: str) -> str:
    """Clean text while preserving date information"""
    if not isinstance(text, str):
        text = str(text)
    
    # Handle typos
    text = re.sub(r'3lst', '31st', text)
    text = re.sub(r'(\d)lst\b', r'\1st', text)
    
    # Normalize whitespace
    text = text.replace('\n', ' ').replace('\t', ' ')
    
    # Remove markdown and formatting
    text = re.sub(r'\*\*|__|`', '', text)
    
    # Remove audit annotations
    text = re.sub(r'\(?\s*(Unaudited|Audited)\s*\)?', ' ', text, flags=re.I)
    
    # Remove extra phrases (but keep ordinal suffixes for date extraction)
    text = re.sub(r'Quarter ended|Six Months ended|Year ended|ended|Refer note \d+', ' ', text, flags=re.I)
    
    # Normalize spaces
    text = re.sub(r'\s+', ' ', text)
    
    return text.strip()

# -----------------------------
# DATE EXTRACTION
# -----------------------------
def extract_all_date_strings(text: str):
    """Extract all potential date strings from text"""
    dates = []
    for match in FULL_DATE_REGEX.finditer(text):
        dates.append(match.group(0).strip())
    return dates

# -----------------------------
# DATE PARSING
# -----------------------------
def parse_date(date_str: str):
    """Parse date string to date object"""
    if not date_str:
        return None
    
    try:
        # Remove ordinal suffixes before parsing
        date_str_clean = re.sub(r'(\d+)(st|nd|rd|th)\b', r'\1', date_str, flags=re.I)
        date_str_clean = date_str_clean.strip()
        
        # Try DD.MM.YYYY or DD/MM/YYYY or DD-MM-YYYY format
        match = re.match(r'^(\d{1,2})[./-](\d{1,2})[./-](\d{2,4})$', date_str_clean)
        if match:
            day, month, year = match.groups()
            year = int(year)
            if year < 100:
                year += 2000 if year < 50 else 1900
            return date(year, int(month), int(day))
        
        # Try DD-Month-YYYY format (30-September-2025)
        match = re.match(r'^(\d{1,2})\s*[-]\s*([A-Za-z]+)\s*[-]\s*(\d{2,4})$', date_str_clean)
        if match:
            day, month_str, year = match.groups()
            month_key = month_str[:4].lower()
            month = MONTH_MAP.get(month_key) or MONTH_MAP.get(month_str.lower())
            if month:
                year = int(year)
                if year < 100:
                    year += 2000 if year < 50 else 1900
                return date(year, month, int(day))
        
        # Try DD Month YYYY format (30 September 2025 or 30 September, 2025)
        match = re.match(r'^(\d{1,2})\s+([A-Za-z]+)\s*,?\s*(\d{4})$', date_str_clean, re.I)
        if match:
            day, month_str, year = match.groups()
            month_key = month_str[:4].lower()
            month = MONTH_MAP.get(month_key) or MONTH_MAP.get(month_str.lower())
            if month:
                return date(int(year), month, int(day))
        
        # Try Month DD YYYY format (September 30 2025 or September 30, 2025)
        match = re.match(r'^([A-Za-z]+)\s+(\d{1,2})\s*,?\s*(\d{4})$', date_str_clean, re.I)
        if match:
            month_str, day, year = match.groups()
            month_key = month_str[:4].lower()
            month = MONTH_MAP.get(month_key) or MONTH_MAP.get(month_str.lower())
            if month:
                return date(int(year), month, int(day))
        
        # Fallback to dateutil
        dt = parser.parse(date_str_clean, dayfirst=True, fuzzy=True)
        return dt.date()
        
    except Exception as e:
        return None

# -----------------------------
# MAIN FUNCTION - SCAN HEADERS AND ROWS
# -----------------------------
def find_date_columns(df, target_dates=TARGET_DATES, max_rows=MAX_SCAN_ROWS, debug=False):
    """
    Scan column headers AND first few rows to find which column contains each target date.
    
    Args:
        df: pandas DataFrame
        target_dates: dict of {name: date_object}
        max_rows: number of rows to scan
        debug: print debug information
    
    Returns:
        dict: {date_name: column_index}
    """
    result = {}
    
    if debug:
        print(f"Scanning column headers and first {max_rows} rows...")
        print(f"Looking for dates: {target_dates}")
        print("="*70 + "\n")
    
    # STEP 1: Scan column headers
    if debug:
        print("STEP 1: Scanning column headers...")
    
    for col_idx, col_name in enumerate(df.columns):
        col_str = str(col_name)
        cleaned = clean_text(col_str)
        
        if debug:
            print(f"Column {col_idx}: '{col_str}'")
        
        date_strings = extract_all_date_strings(cleaned)
        
        if date_strings:
            if debug:
                print(f"  Found date strings: {date_strings}")
            
            for date_str in date_strings:
                parsed_date = parse_date(date_str)
                
                if debug and parsed_date:
                    print(f"  Parsed '{date_str}' → {parsed_date}")
                
                if parsed_date:
                    for date_name, target_date in target_dates.items():
                        if parsed_date == target_date and date_name not in result:
                            result[date_name] = col_idx
                            if debug:
                                print(f"  ✓ MATCH! {date_name} found in column {col_idx}")
    
    # STEP 2: Scan first few rows (for multi-row headers)
    if debug:
        print(f"\nSTEP 2: Scanning first {max_rows} rows...")
    
    for row_idx in range(min(max_rows, len(df))):
        if debug:
            print(f"\nRow {row_idx}:")
        
        for col_idx in range(len(df.columns)):
            cell = df.iloc[row_idx, col_idx]
            
            # Skip NaN and non-string cells
            if pd.isna(cell):
                continue
            
            cell_str = str(cell)
            cleaned = clean_text(cell_str)
            
            if not cleaned:
                continue
            
            # Look for dates
            date_strings = extract_all_date_strings(cleaned)
            
            if date_strings:
                if debug:
                    print(f"  Col {col_idx}: '{cell_str[:60]}...' → found {date_strings}")
                
                for date_str in date_strings:
                    parsed_date = parse_date(date_str)
                    
                    if debug and parsed_date:
                        print(f"    Parsed '{date_str}' → {parsed_date}")
                    
                    if parsed_date:
                        for date_name, target_date in target_dates.items():
                            if parsed_date == target_date and date_name not in result:
                                result[date_name] = col_idx
                                if debug:
                                    print(f"    ✓ MATCH! {date_name} found in column {col_idx}")
        
        # Early exit if all found
        if len(result) == len(target_dates):
            if debug:
                print("\nAll dates found! Stopping scan.")
            break
    
    if debug:
        print("\n" + "="*70)
        print(f"Final result: {result}\n")
    
    return result

# -----------------------------
# CONVENIENCE FUNCTION
# -----------------------------
def get_column_indices(df, max_rows=MAX_SCAN_ROWS, debug=False):
    """
    Get column indices for target dates.
    
    Returns:
        tuple: (this_quarter_column, previous_quarter_column, same_q_last_year_column)
    """
    columns = find_date_columns(df, max_rows=max_rows, debug=debug)
    
    this_quarter_column = columns.get('this_quarter')
    previous_quarter_column = columns.get('previous_quarter')
    same_q_last_year_column = columns.get('same_q_last_year')
    
    return this_quarter_column, previous_quarter_column, same_q_last_year_column

# -----------------------------
# USAGE
# -----------------------------
def get_column_index(df, debug=False):
    """Main function to get column indices"""
    
    # Find columns
    columns = find_date_columns(df, debug=debug)
    
    # Get individual variables
    this_quarter_column, previous_quarter_column, same_q_last_year_column = get_column_indices(df, debug=debug)

    return this_quarter_column, previous_quarter_column, same_q_last_year_column

# -----------------------------
# TEST
# -----------------------------
if __name__ == "__main__":
    # Test with your second table structure
    markdown_file_path = "saved_md_files/aab996e0-1774-4dcd-9c96-576c67805cd0_9.md"
    # 1. Load the markdown file content
    with open(markdown_file_path, 'r') as f:
        text = f.read()
    # 2. Convert text to HTML (ensure 'tables' extension is included)
    table = markdown.markdown(text, extensions=['tables'])
    # 3. Parse HTML with Pandas
    # read_html returns a list of DataFrames, so we take the first one
    df = pd.read_html(io.StringIO(table))[0]
    #print(df.iloc[0:4,:])
    
    # Find columns
    columns = find_date_columns(df, debug=False)
    print("*********************", columns)
    
    # Get individual variables
    this_quarter_column, previous_quarter_column, same_q_last_year_column = get_column_index(df, debug=False)
    print(this_quarter_column)
    print(previous_quarter_column)
    print(same_q_last_year_column)

********************* {}
None
None
None


In [39]:
import re
import pandas as pd

# -----------------------------
# CONFIG
# -----------------------------
TARGET_ROW_PATTERNS = {
    "revenue_from_operations": {
        "must_have": ["revenue from operations"],
        "prefer": ["operations"],
        "exclude": ["total", "other"]
    },
    "total_expenses": {
        "must_have": ["total expense"],
        "prefer": ["total"],
        "exclude": []
    },
    "profit_before_tax": {
        "must_have": ["profit", "tax"],
        "prefer": ["before"],
        "exclude": ["comprehensive", "after"]
    },
    "total_comprehensive_income": {
        "must_have": ["total comprehensive income"],
        "prefer": ["total"],
        "exclude": ["other"]
    }
}

# -----------------------------
# FIXED TEXT CLEANING
# -----------------------------
def clean_row_text(text: str) -> str:
    """Clean text for row matching"""
    if not isinstance(text, str):
        text = str(text)
    
    # Convert to lowercase
    text = text.lower()
    
    # Remove markdown formatting
    text = re.sub(r'\*\*|__|`', '', text)
    
    # Remove parentheses content like (Refer Note 5), (IV), etc
    text = re.sub(r'\([^)]*\)', '', text)
    
    # Remove common prefixes like "a.", "(a)", "i.", "1.", etc
    text = re.sub(r'^[a-z]\.\s*', '', text)
    text = re.sub(r'^\([a-z]\)\s*', '', text)
    text = re.sub(r'^\d+\.\s*', '', text)
    
    # Normalize separators
    text = re.sub(r'[/]+', ' ', text)
    
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text)
    
    return text.strip()

# -----------------------------
# SIMPLIFIED MATCHING
# -----------------------------
def matches_pattern(text: str, pattern_config: dict, debug=False) -> bool:
    """Check if text matches pattern based on keywords."""
    cleaned = clean_row_text(text)
    
    if debug:
        print(f"      Checking: '{text[:60]}...' → cleaned: '{cleaned[:60]}...'")
    
    # Check must-have keywords (ALL required)
    must_have = pattern_config.get("must_have", [])
    for keyword in must_have:
        if keyword not in cleaned:
            if debug:
                print(f"        ✗ Missing required keyword: '{keyword}'")
            return False
    
    # Check exclude keywords (NONE allowed)
    exclude = pattern_config.get("exclude", [])
    for keyword in exclude:
        if keyword in cleaned:
            if debug:
                print(f"        ✗ Contains excluded keyword: '{keyword}'")
            return False
    
    if debug:
        print(f"        ✓ Match!")
    
    return True

# -----------------------------
# MAIN FUNCTION
# -----------------------------
def find_row_indices(df, target_patterns=TARGET_ROW_PATTERNS, 
                     search_columns=5, max_rows=None, debug=True):
    """
    Find row index for first occurrence of each pattern.
    """
    result = {}
    max_rows = max_rows or len(df)
    max_cols = min(search_columns, len(df.columns))
    
    if debug:
        print(f"Searching for row patterns in first {max_cols} columns...")
        print(f"Max rows to search: {max_rows}")
        print("="*70 + "\n")
    
    # For each target pattern, find FIRST occurrence
    for pattern_name, pattern_config in target_patterns.items():
        found = False
        
        if debug:
            print(f"Searching for: {pattern_name}")
            print(f"  Must have ALL: {pattern_config.get('must_have', [])}")
            print(f"  Exclude: {pattern_config.get('exclude', [])}")
        
        # Scan rows (stop at first match)
        for row_idx in range(min(max_rows, len(df))):
            
            if found:
                break
            
            # Check across first N columns
            for col_idx in range(max_cols):
                cell = df.iloc[row_idx, col_idx]
                
                if pd.isna(cell):
                    continue
                
                cell_str = str(cell).strip()
                
                if not cell_str:
                    continue
                
                # Check if this cell matches the pattern
                if matches_pattern(cell_str, pattern_config, debug=debug):
                    result[pattern_name] = row_idx
                    found = True
                    
                    if debug:
                        print(f"  ✓ FOUND at Row {row_idx}, Col {col_idx}")
                        print(f"    Original text: '{cell_str[:80]}'\n")
                    
                    break  # Stop searching columns
        
        if not found and debug:
            print(f"  ✗ Not found\n")
    
    if debug:
        print("="*70)
        print(f"Final result: {result}\n")
    
    return result

# -----------------------------
# CONVENIENCE FUNCTION
# -----------------------------
def get_row_indices(df, search_columns=5, debug=False):
    """
    Get row indices for common financial statement items.
    
    Returns:
        tuple: (revenue_row, expenses_row, pbt_row, comprehensive_income_row)
    """
    rows = find_row_indices(df, search_columns=search_columns, debug=debug)
    
    revenue_row = rows.get('revenue_from_operations')
    expenses_row = rows.get('total_expenses')
    pbt_row = rows.get('profit_before_tax')
    comprehensive_income_row = rows.get('total_comprehensive_income')
    
    return revenue_row, expenses_row, pbt_row, comprehensive_income_row


markdown_file_path = "saved_md_files/2cec8dbb-f9ba-4fc7-b25f-9e019a185852_4.md"
# 1. Load the markdown file content
with open(markdown_file_path, 'r') as f:
    text = f.read()
# 2. Convert text to HTML (ensure 'tables' extension is included)
table = markdown.markdown(text, extensions=['tables'])
# 3. Parse HTML with Pandas
# read_html returns a list of DataFrames, so we take the first one
df = pd.read_html(io.StringIO(table))[0]
#print(df.iloc[0:4,:])


w,x,y,z = get_row_indices(df, search_columns=5, debug=False)

print("\nFinal Variables:")
print(f"revenue_row = {w}")
print(f"expenses_row = {x}")
print(f"pbt_row = {y}")
print(f"comprehensive_income_row = {z}")



Final Variables:
revenue_row = 2
expenses_row = 13
pbt_row = 14
comprehensive_income_row = 26


In [71]:
import re
import pandas as pd
from difflib import get_close_matches

# -----------------------------
# CONFIG - TARGET ROW PATTERNS
# -----------------------------
TARGET_ROW_PATTERNS = {
    "revenue_from_operations": {
        "keywords": ["revenue", "operations"],
        "min_matches": 2,  # At least 1 keyword must match
        "exclude": ["total", "other"]
    },
    "total_expenses": {
        "keywords": ["total", "expense", "expenditure", "before"],
        "min_matches": 2,  # At least 2 keywords must match
        "exclude": []
    },
    "profit_before_tax": {
        "keywords": ["profit", "tax", "before"],
        "min_matches": 2,
        "exclude": ["comprehensive", "after"]
    },
    "total_comprehensive_income": {
        "keywords": ["total", "comprehensive", "income"],
        "min_matches": 3,
        "exclude": ["other"]
    }
}

# -----------------------------
# TEXT CLEANING
# -----------------------------
def clean_row_text(text: str) -> str:
    """Clean text for row matching"""
    if not isinstance(text, str):
        text = str(text)
    
    # Convert to lowercase
    text = text.lower()
    
    # Remove markdown formatting
    text = re.sub(r'\*\*|__|`', '', text)
    
    # Remove parentheses content like (Refer Note 5), (IV), etc
    text = re.sub(r'\([^)]*\)', '', text)
    
    # Remove common prefixes like "a.", "(a)", "i.", "1.", etc
    text = re.sub(r'^[a-z]\.\s*', '', text)
    text = re.sub(r'^\([a-z]\)\s*', '', text)
    text = re.sub(r'^\d+\.\s*', '', text)
    
    # Normalize separators
    text = re.sub(r'[/]+', ' ', text)
    
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text)
    
    return text.strip()

# -----------------------------
# FUZZY MATCHING
# -----------------------------
def matches_pattern(text: str, pattern_config: dict, debug=False) -> bool:
    """Check if text matches pattern based on keyword count with fuzzy matching."""
    cleaned = clean_row_text(text)
    
    if debug:
        print(f"      Checking: '{text[:60]}...'")
        print(f"        Cleaned: '{cleaned[:60]}...'")
    
    # Check exclude keywords first
    exclude = pattern_config.get("exclude", [])
    for keyword in exclude:
        if keyword in cleaned:
            if debug:
                print(f"        ✗ Contains excluded keyword: '{keyword}'")
            return False
    
    # Count matching keywords (with fuzzy matching)
    keywords = pattern_config.get("keywords", [])
    min_matches = pattern_config.get("min_matches", len(keywords))
    
    matched_count = 0
    matched_keywords = []
    
    words_in_text = cleaned.split()
    
    for keyword in keywords:
        # Direct match - check if keyword is in the cleaned text
        if keyword in cleaned:
            matched_count += 1
            matched_keywords.append(keyword)
        # Fuzzy match for typos (e.g., "xpenses" matches "expenses")
        elif len(keyword) > 4:  # Only fuzzy match longer words to avoid false positives
            close = get_close_matches(keyword, words_in_text, n=1, cutoff=0.8)
            if close:
                matched_count += 1
                matched_keywords.append(f"{keyword}≈{close[0]}")
                if debug:
                    print(f"        ~ Fuzzy matched '{keyword}' to '{close[0]}'")
    
    if matched_count >= min_matches:
        if debug:
            print(f"        ✓ Match! ({matched_count}/{min_matches} keywords: {matched_keywords})")
        return True
    else:
        if debug:
            print(f"        ✗ Only {matched_count}/{min_matches} keywords matched: {matched_keywords}")
        return False

# -----------------------------
# MAIN FUNCTION - FIND FIRST OCCURRENCE
# -----------------------------
def find_row_indices(df, target_patterns=TARGET_ROW_PATTERNS, 
                     search_columns=5, max_rows=None, debug=True):
    """
    Find row index for first occurrence of each pattern across multiple columns.
    
    Args:
        df: pandas DataFrame
        target_patterns: dict of pattern configurations
        search_columns: number of columns to search (default 5)
        max_rows: max rows to search (None = search all)
        debug: print debug info
    
    Returns:
        dict: {pattern_name: row_index}
    """
    result = {}
    max_rows = max_rows or len(df)
    max_cols = min(search_columns, len(df.columns))
    
    if debug:
        print(f"Searching for row patterns in first {max_cols} columns...")
        print(f"Max rows to search: {max_rows}")
        print("="*70 + "\n")
    
    # For each target pattern, find FIRST occurrence
    for pattern_name, pattern_config in target_patterns.items():
        found = False
        
        if debug:
            print(f"Searching for: {pattern_name}")
            print(f"  Keywords: {pattern_config.get('keywords', [])}")
            print(f"  Min matches: {pattern_config.get('min_matches', 0)}")
            print(f"  Exclude: {pattern_config.get('exclude', [])}")
        
        # Scan rows (stop at first match)
        for row_idx in range(min(max_rows, len(df))):
            
            if found:
                break
            
            # Check across first N columns
            for col_idx in range(max_cols):
                cell = df.iloc[row_idx, col_idx]
                
                if pd.isna(cell):
                    continue
                
                cell_str = str(cell).strip()
                
                if not cell_str:
                    continue
                
                # Check if this cell matches the pattern
                if matches_pattern(cell_str, pattern_config, debug=debug):
                    result[pattern_name] = row_idx
                    found = True
                    
                    if debug:
                        print(f"  ✓ FOUND at Row {row_idx}, Col {col_idx}")
                        print(f"    Original text: '{cell_str[:80]}'\n")
                    
                    break  # Stop searching columns for this pattern
        
        if not found and debug:
            print(f"  ✗ Not found\n")
    
    if debug:
        print("="*70)
        print(f"Final result: {result}\n")
    
    return result

# -----------------------------
# CONVENIENCE FUNCTION
# -----------------------------
def get_row_indices(df, search_columns=5, debug=False):
    """
    Get row indices for common financial statement items.
    
    Args:
        df: pandas DataFrame
        search_columns: number of columns to search (default 5)
        debug: print debug info
    
    Returns:
        tuple: (revenue_row, expenses_row, pbt_row, comprehensive_income_row)
    """
    rows = find_row_indices(df, search_columns=search_columns, debug=debug)
    
    revenue_row = rows.get('revenue_from_operations')
    expenses_row = rows.get('total_expenses')
    pbt_row = rows.get('profit_before_tax')
    comprehensive_income_row = rows.get('total_comprehensive_income')
    
    return revenue_row, expenses_row, pbt_row, comprehensive_income_row

# -----------------------------
# DIAGNOSTIC HELPER
# -----------------------------
def show_dataframe_content(df, search_columns=5, max_rows=30):
    """Show what's actually in the DataFrame"""
    max_cols = min(search_columns, len(df.columns))
    max_rows = min(max_rows, len(df))
    
    #print("DataFrame Content (First columns and rows):")
    #print("="*70)
    
    for row_idx in range(max_rows):
        has_content = False
        row_str = f"Row {row_idx}: "
        
        for col_idx in range(max_cols):
            cell = df.iloc[row_idx, col_idx]
            if pd.notna(cell) and str(cell).strip():
                has_content = True
                row_str += f"[Col{col_idx}] '{str(cell)[:40]}...' | "
        
        if has_content:
            print(row_str)
    
    print("="*70 + "\n")

# -----------------------------
# USAGE EXAMPLE
# -----------------------------
if __name__ == "__main__":
    # Test datasaved_md_files/40158ce2-884b-4363-9b6d-24beaaa304ce_9.md
    markdown_file_path = "saved_md_files/ea7c5ca5-0cc1-45f1-84af-51c72e6226ab_11.md"
    # 1. Load the markdown file content
    with open(markdown_file_path, 'r') as f:
        text = f.read()
    # 2. Convert text to HTML (ensure 'tables' extension is included)
    table = markdown.markdown(text, extensions=['tables'])
    # 3. Parse HTML with Pandas
    # read_html returns a list of DataFrames, so we take the first one
    df = pd.read_html(io.StringIO(table))[0]
    #print(df.iloc[0:4,:])

    
    # Find rows with detailed debug
    rows = find_row_indices(df, search_columns=5, debug=False)
    
    #print("\nFinal Row Indices:")
    #print(f"revenue_from_operations_row = {rows.get('revenue_from_operations')}")
    #print(f"total_expenses_row = {rows.get('total_expenses')}")
    #print(f"profit_before_tax_row = {rows.get('profit_before_tax')}")
    #print(f"total_comprehensive_income_row = {rows.get('total_comprehensive_income')}")
    
    # Or use convenience function
    print("\n" + "="*70 + "\n")
    revenue_row, expenses_row, pbt_row, comprehensive_income_row = get_row_indices(df, debug=False)
    print("Using convenience function:")
    print(f"revenue_row = {revenue_row}")
    print(f"expenses_row = {expenses_row}")
    print(f"pbt_row = {pbt_row}")
    print(f"comprehensive_income_row = {comprehensive_income_row}")



Using convenience function:
revenue_row = 0
expenses_row = 14
pbt_row = 15
comprehensive_income_row = 24


In [20]:
markdown_file_path = "saved_md_files/3d3c9dfb-537a-491a-a837-23cf5ea1f762_5.md"
# 1. Load the markdown file content
with open(markdown_file_path, 'r') as f:
    text = f.read()
# 2. Convert text to HTML (ensure 'tables' extension is included)
table = markdown.markdown(text, extensions=['tables'])
# 3. Parse HTML with Pandas
# read_html returns a list of DataFrames, so we take the first one
df = pd.read_html(io.StringIO(table))[0]
#print(df.iloc[0:4,:])

Detected Date Columns: {}
This Quarter Column: None
Previous Quarter Column: None
Same Quarter Last Year Column: None
